# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm

#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
# effnet1 = timm.create_model('efficientnet_b1', pretrained=True)
# # M7:block[5], M8:block[6], M9:conv_head
# effnet1

## load config and set logger

In [5]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [6]:
# Setting seed
modeler.set_seed(42)

In [7]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'

TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
PRED_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)

# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/output-disk/domain_adaptation/EfficientNet/exp1/config.yaml'

In [8]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [9]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # source mode
    dev_train_paths = dev_train_source_paths + dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(dev_train_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # source mode
    add_dev_paths = add_dev_source_paths + add_dev_target_paths
    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3026, source_size : 3105, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [10]:
machine_types

['pump', 'gearbox', 'slider', 'ToyCar', 'valve', 'ToyTrain', 'fan']

In [11]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('TRAINING')
    model = Model(n_out=36)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    output_dict, model, pred_df = modeler.run_training(model, dataloaders_dict, writer, optimizer)
    
    # output
    feature_out_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
    model_out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    pred_out_path = f'{PRED_DIR}/{machine_type}_pred.csv'
    # save
    pd.to_pickle(output_dict, feature_out_path)
    torch.save(model.state_dict(), model_out_path)
    pred_df.to_csv(pred_out_path)
    logger.info(f'SAVE SUCCESS : {model_out_path}')
    
    com.toc()

# run

In [12]:
for machine_type in machine_types[0:]:
    run(machine_type, dev_paths)

2021-10-11 13:52:30,450 - 00_train.py - INFO - TARGET MACHINE_TYPE: pump
2021-10-11 13:52:30,451 - 00_train.py - INFO - MAKE DATA_LOADER
2021-10-11 13:52:30,456 - 00_train.py - INFO - TRAINING


use: cuda:0


  0%|                                                                                             | 0/48 [00:00<?, ?it/s]

tensor([[ 0.0911,  0.1600,  0.8319,  ...,  0.1699, -0.0458,  0.0103],
        [-0.2406,  0.1155,  0.3996,  ..., -0.1026,  0.2577,  0.9517],
        [-0.1928, -0.2755, -0.2694,  ...,  0.5607, -0.2178, -0.0352],
        ...,
        [ 1.5385, -0.0535, -0.2774,  ...,  0.4615, -0.1774,  0.5682],
        [-0.2778,  0.0033,  0.2907,  ..., -0.1773, -0.2630,  0.0988],
        [-0.2769, -0.1454,  1.0634,  ...,  1.1577, -0.0469,  1.0685]],
       device='cuda:0', grad_fn=<SiluBackward>)
tensor([[-0.1245, -0.2784,  0.0218,  ..., -0.2554, -0.1430, -0.2283],
        [ 0.0191,  0.1043, -0.2671,  ...,  1.1130,  0.9996, -0.1330],
        [-0.1209,  0.5796, -0.2785,  ...,  0.1115, -0.1641,  0.1537],
        ...,
        [-0.1342,  0.1799,  0.0024,  ..., -0.1901,  0.0942,  0.3922],
        [ 0.3886,  1.8988,  1.0539,  ..., -0.0050,  1.1925, -0.2726],
        [ 0.0885, -0.2778,  0.0257,  ...,  0.6468,  0.3206,  0.1662]],
       device='cuda:0', grad_fn=<SiluBackward>)
tensor([[ 1.4373,  0.3238,  0.1621, 

  2%|█▊                                                                                   | 1/48 [00:01<00:53,  1.13s/it]

...

tensor([[ 0.8012, -0.2023,  0.2873,  ...,  0.2654, -0.2609, -0.0785],
        [ 0.0130, -0.2722,  0.2192,  ..., -0.2527, -0.1778, -0.2766],
        [ 2.5421, -0.1215, -0.1434,  ...,  0.0503, -0.1746, -0.1657],
        ...,
        [-0.2396,  1.3962, -0.0472,  ...,  0.2437, -0.2699,  0.2306],
        [-0.1873,  0.0548,  0.3329,  ...,  1.9074, -0.2438,  0.2018],
        [-0.1446, -0.2630,  0.1039,  ..., -0.0649,  0.8437,  0.0981]],
       device='cuda:0', grad_fn=<SiluBackward>)
tensor([[-0.1907,  0.2185, -0.2766,  ..., -0.1772,  0.0697, -0.2541],
        [ 0.8579,  0.1232, -0.2535,  ...,  1.0160,  0.6333, -0.1032],
        [ 0.2271, -0.2098,  0.2078,  ..., -0.2785, -0.1548,  0.3897],
        ...,
        [-0.1658, -0.2624, -0.2008,  ..., -0.0977, -0.1821, -0.0140],
        [ 0.6442, -0.2784,  0.3454,  ..., -0.1199, -0.2473, -0.2653],
        [-0.2708, -0.2611, -0.1210,  ..., -0.2582, -0.0763,  0.0251]],
       device='cuda:0', grad_fn=<SiluBackward>)
tensor([[ 0.1353,  0.1390, -0.2460, 

 10%|████████▊                                                                            | 5/48 [00:08<01:41,  2.36s/it]

tensor([[-0.2315, -0.1580,  2.7139,  ..., -0.2785,  0.4662, -0.2035],
        [ 1.0904, -0.2650,  0.8061,  ...,  0.0098, -0.2770,  0.3679],
        [-0.2774, -0.1616,  1.0643,  ..., -0.2623,  2.3038, -0.1811],
        ...,
        [-0.0143,  0.0905,  1.1330,  ..., -0.2673,  0.1564, -0.2785],
        [ 0.3277, -0.2056, -0.0758,  ...,  0.0621, -0.0520,  0.2518],
        [-0.2582,  0.9122,  0.3374,  ...,  0.1563, -0.2769, -0.1597]],
       device='cuda:0', grad_fn=<SiluBackward>)
tensor([[-0.1956, -0.0169, -0.1491,  ...,  0.1903,  1.5532, -0.1261],
        [-0.2057, -0.2271,  0.4665,  ..., -0.2137, -0.2648, -0.2563],
        [ 0.0159,  0.1769,  0.6287,  ...,  1.0189, -0.2698,  0.0067],
        ...,
        [-0.0785,  0.8861, -0.2783,  ...,  0.8763,  0.1754, -0.2699],
        [-0.2689,  0.5409, -0.2633,  ..., -0.1730,  0.1193, -0.2087],
        [ 0.0108, -0.2758, -0.0778,  ..., -0.2769,  1.2287,  0.7737]],
       device='cuda:0', grad_fn=<SiluBackward>)
tensor([[-0.2388,  1.4761,  0.8010, 

 12%|██████████▋                                                                          | 6/48 [00:15<01:48,  2.58s/it]


KeyboardInterrupt: 

In [ ]:
#run(machine_types[0], dev_paths)

In [ ]:
import gc
gc.collect()